In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import numpy.random as npr
import random

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import backend as K
from keras.optimizers import Adam
from keras_nlp.layers import PositionEmbedding

In [3]:
seed = 428

np.random.seed(seed)
tf.random.set_seed(seed)
random.seed(seed)

In [4]:
def get_masked_input_and_labels(encoded_texts, n_cat):
    # For each sentence, mask each word one-by-one

    encoded_texts_masked = []
    y_labels = []

    for encoded_text in encoded_texts:
        for i in range(len(encoded_text)):
            encoded_text_masked = np.copy(encoded_text)
            y_label = encoded_text_masked[i]
            encoded_texts_masked.append(np.delete(encoded_text_masked, i))
            y_labels.append(np.array([y_label]))

    return np.array(encoded_texts_masked), np.array(y_labels)

In [5]:
#### K = number of countries = number of capitals
#### S = number of words used by both topics
#### L = sentence length
#### q1, q2 = probability of having 1 or 2 pairs
#### embed_dim = dimension of embeddings
#### n_sentences = number of training sentences

def train_model(K, S, L, q1, q2, embed_dim, n_sentences):
    
    countries = ['country_' + str(i) for i in range(K)]
    capitals = ['capital_' + str(i) for i in range(K)]
    randoms = ['random_' + str(i) for i in range(S)]

    vocabs = countries + capitals + randoms
    vocab_map = {}

    for i in range(len(vocabs)):
        vocab_map[vocabs[i]] = i
        
    sentences = []
    sentences_number = []
    
    q0 = 1 - q1 - q2

    for i in range(n_sentences):

        sentence = []

        temp2 = npr.uniform()
        
        if temp2 <= q0:
            n_pairs = 0
        elif temp2 <= q0 + q1:
            n_pairs = 1
        else:
            n_pairs = 2
        
        pairs = np.random.choice(np.arange(K), n_pairs, replace = False)
        for pair in pairs:
            sentence.append(countries[pair])
            sentence.append(capitals[pair])        
        sentence += list(np.random.choice(randoms, L - 2 * n_pairs, replace = False))

        sentence_number = [vocab_map[i] for i in sentence]
        sentences.append(sentence)
        sentences_number.append(sentence_number)
        
    x_train = np.array(sentences_number)
    n_cat = len(vocab_map)
    x_masked_train, y_masked_labels_train = get_masked_input_and_labels(x_train, n_cat)
    
    callback = keras.callbacks.EarlyStopping(monitor = 'val_loss', patience = 5, restore_best_weights = True)
    inputs = layers.Input((x_masked_train.shape[1],), dtype=tf.int64)
    word_embeddings = layers.Embedding(n_cat, embed_dim, name="word_embedding")(inputs)
    encoder_output = layers.GlobalAveragePooling1D()(word_embeddings)
    mlm_output = layers.Dense(n_cat, name="mlm_cls", activation="softmax", use_bias=False)(encoder_output)
    mlm_model = keras.Model(inputs = inputs, outputs = mlm_output)
    adam = Adam()
    mlm_model.compile(loss='sparse_categorical_crossentropy', optimizer=adam)

    history = mlm_model.fit(x_masked_train, y_masked_labels_train,
                        validation_split = 0.5, callbacks = [callback], 
                        epochs=500, batch_size=128, verbose=0)
    
    return sentences, vocab_map, mlm_model

In [6]:
def get_acc_prob(K, S, L, q1, q2, embed_dim, n_sentences, n_samples):
    
    sentences, vocab_map, current_model = train_model(K, S, L, q1, q2, embed_dim, n_sentences)

    acc_countries = []
    prob_countries = []

    for _ in range(n_samples):
        sentence = []
        random_countries = np.random.choice(np.arange(K), int(L/2), replace = False)
        for random_country in random_countries:
            sentence.append('country_' + str(random_country))
            sentence.append('capital_' + str(random_country))
        sentence = sentence[:-1]
        sentence_number = [vocab_map[i] for i in sentence]
        temp = keras.backend.function(inputs = current_model.layers[0].input, outputs = current_model.layers[-1].output) \
            (np.array(sentence_number).reshape(1,len(sentence_number)))
        actual = vocab_map['capital_' + str(random_countries[-1])]
        acc_countries.append(1 if np.argsort(-1 * temp)[0][0] == actual else 0)
        prob_countries.append(temp[0][vocab_map['capital_' + str(random_countries[-1])]])
        

    return sentences, current_model, vocab_map, (np.mean(acc_countries), np.mean(prob_countries))

In [7]:
K = 10 # number of countries
L = 8 # sentence length
S = 20 # number of words used by both topics
embed_dim = 100 # CBOW embedding dimension
n_sentences = 50000 # number of sentences in the training set
n_samples = 1000

In [9]:
q0 = 0 # probability of having 0 pairs
q1 = 1 # probability of having 1 pair
q2 = 0 # probability of having 2 pairs

accs = 0
probs = 0

for _ in range(10):
    sentences, mlm_model, vocab_map, acc_c \
        = get_acc_prob(K, S, L, q1, q2, embed_dim, n_sentences, n_samples)
    
    print(acc_c)
    
    accs += acc_c[0]/10
    probs += acc_c[1]/10
    
print((accs, probs))

(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 5.31222e-40)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 2.2890677e-37)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 2.2943799010748445e-38)


In [10]:
q0 = 0 # probability of having 0 pairs
q1 = 0 # probability of having 1 pair
q2 = 1 # probability of having 2 pairs

accs = 0
probs = 0

for _ in range(10):
    sentences, mlm_model, vocab_map, acc_c \
        = get_acc_prob(K, S, L, q1, q2, embed_dim, n_sentences, n_samples)
    
    print(acc_c)
    
    accs += acc_c[0]/10
    probs += acc_c[1]/10
    
print((accs, probs))

(0.0, 1.5488775e-21)
(0.0, 1.1768545e-23)
(0.0, 2.5465403e-21)
(0.0, 5.923689e-17)
(0.0, 3.4913973e-17)
(0.0, 1.4355865e-20)
(0.0, 3.4501077e-21)
(0.0, 2.1585401e-19)
(0.0, 5.2184864e-18)
(0.0, 3.4270036e-22)
(0.0, 9.960746161072905e-18)


In [11]:
q0 = 1/2 # probability of having 0 pairs
q1 = 1/2 # probability of having 1 pair
q2 = 0 # probability of having 2 pairs

accs = 0
probs = 0

for _ in range(10):
    sentences, mlm_model, vocab_map, acc_c \
        = get_acc_prob(K, S, L, q1, q2, embed_dim, n_sentences, n_samples)
    
    print(acc_c)
    
    accs += acc_c[0]/10
    probs += acc_c[1]/10
    
print((accs, probs))

(0.996, 0.89421016)
(0.976, 0.83682203)
(0.985, 0.88399744)
(0.99, 0.90350837)
(0.955, 0.75299823)
(0.994, 0.88997465)
(0.981, 0.88729113)
(0.996, 0.92090416)
(0.993, 0.9272482)
(0.997, 0.85806125)
(0.9863000000000001, 0.875501561164856)


In [12]:
q0 = 1/2 # probability of having 0 pairs
q1 = 0 # probability of having 1 pair
q2 = 1/2 # probability of having 2 pairs

accs = 0
probs = 0

for _ in range(10):
    sentences, mlm_model, vocab_map, acc_c \
        = get_acc_prob(K, S, L, q1, q2, embed_dim, n_sentences, n_samples)
    
    print(acc_c)
    
    accs += acc_c[0]/10
    probs += acc_c[1]/10
    
print((accs, probs))

(1.0, 1.0)
(1.0, 1.0)
(1.0, 1.0)
(1.0, 1.0)
(1.0, 1.0)
(1.0, 1.0)
(1.0, 1.0)
(1.0, 1.0)
(1.0, 1.0)
(1.0, 1.0)
(0.9999999999999999, 0.9999999999999999)


In [15]:
q0 = 0 # probability of having 0 pairs
q1 = 1/2 # probability of having 1 pair
q2 = 1/2 # probability of having 2 pairs

accs = 0
probs = 0

for _ in range(10):
    sentences, mlm_model, vocab_map, acc_c \
        = get_acc_prob(K, S, L, q1, q2, embed_dim, n_sentences, n_samples)
    
    print(acc_c)
    
    accs += acc_c[0]/10
    probs += acc_c[1]/10
    
print((accs, probs))

(1.0, 0.9999624)
(1.0, 0.9999607)
(1.0, 0.99996996)
(1.0, 0.99997425)
(1.0, 0.9999568)
(1.0, 0.9999692)
(1.0, 0.99997056)
(1.0, 0.9999715)
(1.0, 0.9999614)
(1.0, 0.99996287)
(0.9999999999999999, 0.9999659597873689)


In [14]:
q0 = 1/3 # probability of having 0 pairs
q1 = 1/3 # probability of having 1 pair
q2 = 1/3 # probability of having 2 pairs

accs = 0
probs = 0

for _ in range(10):
    sentences, mlm_model, vocab_map, acc_c \
        = get_acc_prob(K, S, L, q1, q2, embed_dim, n_sentences, n_samples)
    
    print(acc_c)
    
    accs += acc_c[0]/10
    probs += acc_c[1]/10
    
print((accs, probs))

(1.0, 0.9999999)
(1.0, 0.99999887)
(1.0, 0.9999999)
(1.0, 0.99999976)
(1.0, 0.9999999)
(1.0, 0.9999999)
(1.0, 0.9999999)
(1.0, 0.9999999)
(1.0, 0.99999887)
(1.0, 0.9999999)
(0.9999999999999999, 0.9999996662139893)
